In [11]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from datetime import timedelta
from datetime import datetime
import scipy.stats as stats

import requests as r
import pandas as pd
import seaborn as s
import numpy as np

import postgres
import gmaps
import json
import math

In [18]:
import importlib
importlib.reload(postgres.config)

<module 'config' from '/media/storage/College/S3/Github/DublinBus/Analytics/config.py'>

In [13]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Import Data

In [19]:
data = postgres.query("SELECT * FROM combined WHERE lineid='145';")

Connection failed with error:
OperationalError('could not translate host name "(\'137.43.49.60\'" to address: Name or service not known\ncould not translate host name ")" to address: Name or service not known\n',)


## Train Model

## Evaluate Model

## Changes